In [53]:
import pandas as pd
from pathlib import Path
import requests
from bs4 import BeautifulSoup

In [71]:
#Import danych
data_path = Path() / "data"
dane1 = pd.read_csv(data_path / "arq_casco_comp.csv", sep= ';')

C:\Users\Dawid\AppData\Local\Temp\ipykernel_22324\3564885659.py:3: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  dane1 = pd.read_csv(data_path / "arq_casco_comp.csv", sep= ';')


In [73]:
dane1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3390758 entries, 0 to 3390757
Data columns (total 22 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   COD_TARIF   int64 
 1   REGIAO      object
 2   COD_MODELO  object
 3   ANO_MODELO  int64 
 4   SEXO        object
 5   IDADE       int64 
 6   EXPOSICAO1  object
 7   PREMIO1     object
 8   EXPOSICAO2  int64 
 9   PREMIO2     int64 
 10  IS_MEDIA    object
 11  FREQ_SIN1   int64 
 12  INDENIZ1    int64 
 13  FREQ_SIN2   int64 
 14  INDENIZ2    int64 
 15  FREQ_SIN3   int64 
 16  INDENIZ3    int64 
 17  FREQ_SIN4   int64 
 18  INDENIZ4    int64 
 19  FREQ_SIN9   int64 
 20  INDENIZ9    int64 
 21  ENVIO       object
dtypes: int64(15), object(7)
memory usage: 569.1+ MB


In [74]:
dane_praca = dane1

In [75]:
#Usuniecie wierszy gdzie REGIAO jest równe " ."
dane_praca.isna().sum()
to_remove = dane_praca["REGIAO"] == " ."
dane_praca = dane_praca[~to_remove]

In [76]:
#Zamiana wartosci w kolumnie REGIAO na obiekt typu int
dane_praca["REGIAO"] = pd.to_numeric(dane_praca["REGIAO"])
pd.unique(dane_praca["REGIAO"])

C:\Users\Dawid\AppData\Local\Temp\ipykernel_22324\1952280371.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dane_praca["REGIAO"] = pd.to_numeric(dane_praca["REGIAO"])


array([11,  2, 23, 39, 13, 17, 38, 12,  6, 10,  5, 22,  8, 26, 18,  9, 28,
       14,  4, 16,  7, 30,  1, 41, 20, 15,  3, 37, 35, 21, 99, 27, 40, 25,
       36, 31, 19, 29, 24,  0, 32, 33, 34], dtype=int64)

In [77]:
#Zmiana nazw kolumn
brvehins = dane_praca.rename(columns={"SEXO": "Gender",
                                "COD_MODELO": "FullVehCode",
                                 "ANO_MODELO": "VehYear",
                                 "IDADE":"DrivAge",
                                 "COD_TARIF":"VehCode",
                                 "REGIAO":"State",
                                 "IS_MEDIA":"SumInsAvg",
                                 "EXPOSICAO1":"ExposTotal",
                                 "PREMIO1":"PremTotal",
                                 "EXPOSICAO2":"ExposFireRob",
                                 "PREMIO2":"PremFireRob",
                                 "FREQ_SIN1":"ClaimNbRob",
                                 "INDENIZ1":"ClaimAmountRob",
                                 "FREQ_SIN2":"ClaimNbPartColl",
                                 "INDENIZ2":"ClaimAmountPartColl",
                                 "FREQ_SIN3":"ClaimNbTotColl",
                                 "INDENIZ3":"ClaimAmountTotColl",
                                 "FREQ_SIN4":"ClaimNbFire",
                                 "INDENIZ4":"ClaimAmountFire",
                                 "FREQ_SIN9":"ClaimNbOther",
                                 "INDENIZ9":"ClaimAmountOther"
                                 })
brvehins.head()

,VehCode,State,FullVehCode,VehYear,Gender,DrivAge,ExposTotal,PremTotal,ExposFireRob,PremFireRob,...,ClaimAmountRob,ClaimNbPartColl,ClaimAmountPartColl,ClaimNbTotColl,ClaimAmountTotColl,ClaimNbFire,ClaimAmountFire,ClaimNbOther,ClaimAmountOther,ENVIO
0,9,11,515153-8,2014,J,0,"16,8219174854457","20984,1537723541",0,0,...,0,0,0,0,0,0,0,0,0,2021A
1,1,2,025081-3,2006,F,5,"0,734246578067541","421,130994796753",0,0,...,0,0,0,0,0,0,0,0,0,2021A
2,1,23,004318-4,2014,F,2,"0,504109561443329","309,683074951172",0,0,...,0,0,0,0,0,0,0,0,0,2021A
3,1,39,004354-0,2011,F,5,"0,402739733457565","257,049194335938",0,0,...,0,0,0,0,0,0,0,0,0,2021A
4,1,13,515021-3,1999,M,5,"0,506849318742752","947,028137207031",0,0,...,0,0,0,0,0,0,0,0,0,2021A


In [78]:
#Wiek auta
brvehins["VehYear"] = 2020 - brvehins["VehYear"]

In [79]:
#Wiek kierowcy
wiek = pd.read_csv(data_path / "auto_idade.csv", sep=";")
brvehins = brvehins.merge(wiek, left_on="DrivAge", right_on="Kod", how = "left")
brvehins = brvehins.drop(['DrivAge','Kod'], axis=1)


In [80]:
#Kolumna VehCode
vehcode = pd.read_csv(data_path / 'auto_cat.csv', encoding='unicode_escape', sep=';')
brvehins = brvehins.merge(vehcode, left_on='VehCode', right_on=vehcode['Kod '], how = "left")
brvehins = brvehins.drop(['Kod '], axis=1)


In [81]:
#Wgrywanie tabeli z Wikipedii - nazwy stanów
wikiurl = 'https://en.wikipedia.org/wiki/Federative_units_of_Brazil'
response=requests.get(wikiurl)
print(response.status_code)
soup = BeautifulSoup(response.text, 'html.parser')
indiatable=soup.find('table',{'class':"wikitable"})
#Zamiana na dataframe
state_full=pd.read_html(str(indiatable))
state_full=pd.DataFrame(state_full[0])
print(state_full.head())


200
  Flag and name Code     Capital Largest city  Area (km2)[20]  \
0          Acre   AC  Rio Branco   Rio Branco          164123   
1       Alagoas   AL      Maceió       Maceió           27779   
2         Amapá   AP      Macapá       Macapá          142829   
3      Amazonas   AM      Manaus       Manaus         1559159   
4         Bahia   BA    Salvador     Salvador          564733   

   Population (May 2023)[21]  Density (per km2, 2022)  \
0                     830018                     6.34   
1                    3127683                   125.52   
2                     733759                     2.63   
3                    3941613                     2.58   
4                   14141626                    30.52   

   GDP (R$ millions, 2022)[22]  HDI (2022)[23]  
0                        21000           0.719  
1                        73000           0.683  
2                         9000           0.740  
3                       103000           0.733  
4                

In [82]:
#State AB and full name
states_full = state_full[["Flag and name", "Code"]]
states_full = pd.DataFrame(states_full)
states_full = states_full.rename(columns={"Flag and name": "State"})
states_full

,State,Code
0,Acre,AC
1,Alagoas,AL
2,Amapá,AP
3,Amazonas,AM
4,Bahia,BA
5,Ceará,CE
6,Distrito Federal,DF
7,Espírito Santo,ES
8,Goiás,GO
9,Maranhão,MA


In [83]:
#Kolumna State
state = pd.read_csv(data_path / 'auto_reg.csv', encoding='unicode_escape', sep = ';')
state[['StateAb','Area']] = state['State_decode'].str.split(' - ', expand=True)
state = state.drop(['State_decode'], axis=1)
brvehins = brvehins.merge(state, left_on='State', right_on='Kod', how = "left")
brvehins = brvehins.merge(states_full, left_on="StateAb", right_on="Code", how = "left")
brvehins = brvehins.drop(['Kod', 'Code'], axis = 1)
brvehins


,VehCode,State_x,FullVehCode,VehYear,Gender,ExposTotal,PremTotal,ExposFireRob,PremFireRob,SumInsAvg,...,ClaimNbFire,ClaimAmountFire,ClaimNbOther,ClaimAmountOther,ENVIO,DrivAge_decode,VehCode_decode,StateAb,Area,State_y
0,9,11,515153-8,6,J,"16,8219174854457","20984,1537723541",0,0,"24936,1741912814",...,0,0,0,0,2021A,NaN,Inne,SP,Met. de São Paulo,São Paulo
1,1,2,025081-3,14,F,"0,734246578067541","421,130994796753",0,0,"11707,3506504372",...,0,0,0,0,2021A,Powyzej 55,Auto krajowe,RS,Demais regiões,Rio Grande do Sul
2,1,23,004318-4,6,F,"0,504109561443329","309,683074951172",0,0,"17903,9991278803",...,0,0,0,0,2021A,Miedzy 26 a 35,Auto krajowe,PE,Pernambuco,Pernambuco
3,1,39,004354-0,9,F,"0,402739733457565","257,049194335938",0,0,"22636,9999743224",...,0,0,0,0,2021A,Powyzej 55,Auto krajowe,GO,Goiás,Goiás
4,1,13,515021-3,21,M,"0,506849318742752","947,028137207031",0,0,"53464,8870965855",...,0,0,0,0,2021A,Powyzej 55,Auto krajowe,SP,Ribeirão Preto e Demais Mun. de Campinas,São Paulo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3390613,1,37,001316-1,7,M,"0,504109561443329","667,125671386719",0,0,"29488,9993375154",...,0,0,0,0,2021A,Miedzy 18 a 25,Auto krajowe,MS,Mato Grosso do Sul,Mato Grosso do Sul
3390614,5,35,825074-0,0,M,"0,484931498765945","782,475402832031",0,0,"36826,9999186927",...,0,0,0,0,2021A,Powyzej 55,Motocykl (krajowy i importowany),AC,Acre,Acre
3390615,1,2,004276-5,15,M,"0,24657533923164","12,9051642417908",0,0,"13284,4776081873",...,0,0,1,519,2021A,Miedzy 46 a 55,Auto krajowe,RS,Demais regiões,Rio Grande do Sul
3390616,3,23,022094-9,10,J,"0,504109561443329","25,7920513153076",0,0,0,...,0,0,0,0,2021A,NaN,Pick-up (krajowy i importowany),PE,Pernambuco,Pernambuco


In [84]:
#Kolumna FullVehCode
FullVehCode = pd.read_csv(data_path / 'auto2_vei.csv', encoding='unicode_escape', sep = ';')
brvehins = brvehins.merge(FullVehCode, left_on='FullVehCode', right_on='Kod', how = 'left')
brvehins = brvehins.drop(['Kod'], axis = 1)
brvehins

,VehCode,State_x,FullVehCode,VehYear,Gender,ExposTotal,PremTotal,ExposFireRob,PremFireRob,SumInsAvg,...,ClaimAmountFire,ClaimNbOther,ClaimAmountOther,ENVIO,DrivAge_decode,VehCode_decode,StateAb,Area,State_y,Auto
0,9,11,515153-8,6,J,"16,8219174854457","20984,1537723541",0,0,"24936,1741912814",...,0,0,0,2021A,NaN,Inne,SP,Met. de São Paulo,São Paulo,25-420 Constellation 6x2 2p (Diesel)(E5)
1,1,2,025081-3,14,F,"0,734246578067541","421,130994796753",0,0,"11707,3506504372",...,0,0,0,2021A,Powyzej 55,Auto krajowe,RS,Demais regiões,Rio Grande do Sul,Renault - Clio Authentique 1.0 8V 3p
2,1,23,004318-4,6,F,"0,504109561443329","309,683074951172",0,0,"17903,9991278803",...,0,0,0,2021A,Miedzy 26 a 35,Auto krajowe,PE,Pernambuco,Pernambuco,GM - Chevrolet - Celta Life 1.0 MPFI 8V FlexPo...
3,1,39,004354-0,9,F,"0,402739733457565","257,049194335938",0,0,"22636,9999743224",...,0,0,0,2021A,Powyzej 55,Auto krajowe,GO,Goiás,Goiás,GM - Chevrolet - Meriva Maxx 1.4 MPFI 8V ECONO...
4,1,13,515021-3,21,M,"0,506849318742752","947,028137207031",0,0,"53464,8870965855",...,0,0,0,2021A,Powyzej 55,Auto krajowe,SP,Ribeirão Preto e Demais Mun. de Campinas,São Paulo,VOLKSWAGEN - 14-170 BT 3-Eixos 2p (diesel)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3390613,1,37,001316-1,7,M,"0,504109561443329","667,125671386719",0,0,"29488,9993375154",...,0,0,0,2021A,Miedzy 18 a 25,Auto krajowe,MS,Mato Grosso do Sul,Mato Grosso do Sul,Punto ESSENCE 1.6 Flex 16V 5p
3390614,5,35,825074-0,0,M,"0,484931498765945","782,475402832031",0,0,"36826,9999186927",...,0,0,0,2021A,Powyzej 55,Motocykl (krajowy i importowany),AC,Acre,Acre,GSX-S 750
3390615,1,2,004276-5,15,M,"0,24657533923164","12,9051642417908",0,0,"13284,4776081873",...,0,1,519,2021A,Miedzy 46 a 55,Auto krajowe,RS,Demais regiões,Rio Grande do Sul,GM - Chevrolet - Corsa Sed Classic Spirit 1.6 ...
3390616,3,23,022094-9,10,J,"0,504109561443329","25,7920513153076",0,0,0,...,0,0,0,2021A,NaN,Pick-up (krajowy i importowany),PE,Pernambuco,Pernambuco,Mitsubishi - L200 OUTDOOR HPE 2.5 4x4 CD T.Die...


In [85]:
brvehins.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3390618 entries, 0 to 3390617
Data columns (total 27 columns):
 #   Column               Dtype 
---  ------               ----- 
 0   VehCode              int64 
 1   State_x              int64 
 2   FullVehCode          object
 3   VehYear              int64 
 4   Gender               object
 5   ExposTotal           object
 6   PremTotal            object
 7   ExposFireRob         int64 
 8   PremFireRob          int64 
 9   SumInsAvg            object
 10  ClaimNbRob           int64 
 11  ClaimAmountRob       int64 
 12  ClaimNbPartColl      int64 
 13  ClaimAmountPartColl  int64 
 14  ClaimNbTotColl       int64 
 15  ClaimAmountTotColl   int64 
 16  ClaimNbFire          int64 
 17  ClaimAmountFire      int64 
 18  ClaimNbOther         int64 
 19  ClaimAmountOther     int64 
 20  ENVIO                object
 21  DrivAge_decode       object
 22  VehCode_decode       object
 23  StateAb              object
 24  Area                 obj

In [89]:
brvehins_done = brvehins.iloc[:,[4,21,3,22,26,24,25,23,5,7,6,8,9,10,11,12,13,14,15,16,17,18,19]]
brvehins_done = brvehins_done.rename(columns={"DrivAge_decode": "DrivAge",
                              "VehCode_decode": "VehCode",
                              "Auto": "FullVehCode",
                              "State_y": "State"})


In [91]:
brvehins_done.isna().sum()

Gender                      0
DrivAge                658240
VehYear                     0
VehCode                     0
FullVehCode            134750
Area                    36074
State                   36074
StateAb                 36074
ExposTotal                  0
ExposFireRob                0
PremTotal                   0
PremFireRob                 0
SumInsAvg                   0
ClaimNbRob                  0
ClaimAmountRob              0
ClaimNbPartColl             0
ClaimAmountPartColl         0
ClaimNbTotColl              0
ClaimAmountTotColl          0
ClaimNbFire                 0
ClaimAmountFire             0
ClaimNbOther                0
ClaimAmountOther            0
dtype: int64